## Training ExtraTreesClassifier and XGBoost

### 1. Data Preparation and cleaning

In [80]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer

In [81]:
df = pd.read_csv("fitness_class_2212.csv")

In [82]:
df.weight = df.weight.fillna(df.weight.median()) #For ExtraTreesClassifier, there is a need to handle missing values, not for XGBoost
df.drop('booking_id', axis=1, inplace=True)  # drop the booking_id column as it's irrelevant to the analysis
df.days_before = df.days_before.str.replace(r' days$', '', regex=True) # Remove ' days'
df.days_before = pd.to_numeric(df.days_before, errors='coerce').astype(int) # Convert the cleaned values to integers

df.day_of_week = df.day_of_week.replace({'Wednesday': 'Wed', 'Monday': 'Mon'})  # Replace 'Wednesday' with 'Wed'
df.day_of_week = df.day_of_week.str.replace(r'Fri\.$', 'Fri', regex=True)  # Use raw string to remove period from 'Fri.'
df.category = df.category.replace('-', 'Unknown')  # Replace with "Unknown" instead of NaN to prevent information loss

### 2. Splitting the data, extracting target variables, dropping target column and transforming features

In [83]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train = df_train['attended'].values
y_val = df_val['attended'].values
y_test = df_test['attended'].values

del df_train['attended']
del df_val['attended']
del df_test['attended']

### 3a. Training ExtraTreesClassifier
- This class implements a meta estimator that fits a number of randomized decision trees (a.k.a. extra-trees) on various sub-samples of the dataset and uses averaging to improve the predictive accuracy and control over-fitting. https://scikit-learn.org/dev/modules/generated/sklearn.ensemble.ExtraTreesClassifier.html

In [84]:
dv = DictVectorizer(sparse=False)

train_dicts = df_train.to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val.to_dict(orient='records')
X_val = dv.transform(val_dicts)

test_dicts = df_test.to_dict(orient='records')
X_test = dv.transform(test_dicts)

In [85]:
et = ExtraTreesClassifier(n_estimators=100, random_state=1)
et.fit(X_train, y_train)

ExtraTreesClassifier(random_state=1)

In [86]:
#Validating the model

y_val_pred = et.predict_proba(X_val)[:, 1]
val_auc = roc_auc_score(y_val, y_val_pred)
print("Validation AUC:", val_auc)

Validation AUC: 0.7058363858363859


#### 3b. Tuning the parameters with GridSearchCV (to search for the best combination of parameters) 

In [87]:
from sklearn.model_selection import GridSearchCV

In [88]:
# Defining the hyperparameter grid

param_grid = {
    "n_estimators": [50, 100, 150, 200],
    "max_depth": [10, 20, 30, None],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "max_features": ["sqrt", "log2", None]
}

In [89]:
et = ExtraTreesClassifier(random_state=1) #Initializing the classifier

In [90]:
# Performing GridSearchCV

grid_search = GridSearchCV(
    estimator=et,
    param_grid=param_grid,
    scoring="roc_auc",
    cv=5,
    verbose=2,
    n_jobs=-1
)

In [91]:
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 432 candidates, totalling 2160 fits
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.1s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.1s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.1s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.2s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.2s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=150; total time=   0.3s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=150; total time=   0.2s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=150

GridSearchCV(cv=5, estimator=ExtraTreesClassifier(random_state=1), n_jobs=-1,
             param_grid={'max_depth': [10, 20, 30, None],
                         'max_features': ['sqrt', 'log2', None],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 150, 200]},
             scoring='roc_auc', verbose=2)

In [92]:
print("Best Parameters:", grid_search.best_params_)
print("Best Validation AUC:", grid_search.best_score_)

Best Parameters: {'max_depth': 10, 'max_features': None, 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 50}
Best Validation AUC: 0.8147076923076924


##### Explanation of result:
- `max_depth=10`: This limits the depth of each tree to 10 levels, balancing model complexity and overfitting. A smaller depth prevents the model from memorizing the training data.
- `max_features=None`: The model considers all features when determining splits, potentially increasing performance because there are only 6 features.
- `min_samples_leaf=2`: Each leaf node must have at least 2 samples, preventing splits that would result in very small or pure leaf nodes. This can help reduce overfitting.
- `min_samples_split=10`: A node must have at least 10 samples to be considered for splitting, further controlling overfitting.
- `n_estimators=50`: The model uses 50 trees in the ensemble, which is reasonable for your dataset size (1500 rows). More trees would increase training time but may not significantly improve performance.

#### 3c. Training the model with the Best Parameters.

In [93]:
best_et = ExtraTreesClassifier(
    max_depth=10,
    max_features=None,
    min_samples_leaf=2,
    min_samples_split=10,
    n_estimators=50,
    random_state=1
)

best_et.fit(X_train, y_train)

ExtraTreesClassifier(max_depth=10, max_features=None, min_samples_leaf=2,
                     min_samples_split=10, n_estimators=50, random_state=1)

In [94]:
# Evaluating on validation dataset

y_val_pred = best_et.predict_proba(X_val)[:, 1]
val_auc = roc_auc_score(y_val, y_val_pred)
print("Validation AUC:", val_auc)

Validation AUC: 0.8179731379731381


In [95]:
# Evaluate on test set
y_test_pred = best_et.predict_proba(X_test)[:, 1]
test_auc = roc_auc_score(y_test, y_test_pred)
print("Test AUC:", test_auc)

Test AUC: 0.7448615508317


#### The model appears to be slightly overfitting to the training and validation data given that the `Test AUC: 0.745` is lower than the `Validation AUC: 0.818`.

#### 3d. Cross Validation

In [96]:
from sklearn.model_selection import cross_val_score

cv_scores = cross_val_score(
    best_et,
    X_train,
    y_train,
    cv=5,
    scoring="roc_auc"
)

print("Cross-Validation AUC:", np.mean(cv_scores), "+-", np.std(cv_scores))

Cross-Validation AUC: 0.8147076923076924 +- 0.02596120911472005


#### `The Cross-Validation AUC: 0.815` does indicate the the model is performing consistently across folds. Moving on...

### 4a. Training, tuning and evaluating an XGBoost model
- XG Boost: XGBoost is an optimized distributed gradient boosting library designed to be highly efficient, flexible and portable. It implements machine learning algorithms under the Gradient Boosting framework. https://www.nvidia.com/en-us/glossary/xgboost/

In [97]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [98]:
import xgboost as xgb

In [99]:
# Creating DMatrix for training, validation, and testing

dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)
dtest = xgb.DMatrix(X_test, label=y_test)

In [100]:
# Setting baseline parameters for the model

xgb_params = {
    "objective": "binary:logistic",
    "eval_metric": "auc",
    "max_depth": 10,
    "eta": 0.1,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "random_state": 1,
}

In [101]:
# Training, and evaluating the model by using an evaluation watchlist to monitor the performance

watchlist = [(dtrain, "train"), (dval, "val")]

model = xgb.train(
    xgb_params,
    dtrain,
    num_boost_round=100,
    evals=watchlist,
    early_stopping_rounds=10,
    verbose_eval=10,
)

[0]	train-auc:0.88321	val-auc:0.76874
[10]	train-auc:0.94066	val-auc:0.82056
[20]	train-auc:0.95743	val-auc:0.81038


In [102]:
# Evaluating on Test dataset

y_test_pred = model.predict(dtest)

test_auc = roc_auc_score(y_test, y_test_pred)
print("XGBoost Test AUC:", test_auc)

XGBoost Test AUC: 0.747449620583949


#### An XGBoost Test AUC of 0.747 is slightly better than that of the ExtraTreesClassifier Test AUC of 0.745 but it's still not a significant improvement.

#### The imbalance in the dataset appears to be affecting the performance of XGBoost

#### 4b. Ensemble Models: Combine predictions from multiple models, such as XGBoost, ExtraTrees, and LightGBM, using a soft voting approach. This often improves generalization.

In [103]:
from sklearn.ensemble import VotingClassifier

ensemble_model = VotingClassifier(
    estimators=[
        ('xgb', best_xgb),
        ('et', best_et),
    ],
    voting='soft'
)

ensemble_model.fit(X_train, y_train)

# Evaluate on test set
y_test_pred = ensemble_model.predict_proba(X_test)[:, 1]
test_auc = roc_auc_score(y_test, y_test_pred)
print("Ensemble Test AUC:", test_auc)

Ensemble Test AUC: 0.7592341323684607


In [104]:
# Fine tuning the Ensemble model

ensemble_model = VotingClassifier(
    estimators=[
        ('xgb', best_xgb),
        ('et', best_et),
    ],
    voting='soft',
    weights=[2, 1]  # Give more weight to XGBoost if it performs better
)

ensemble_model.fit(X_train, y_train)

# Evaluate on test set
y_test_pred = ensemble_model.predict_proba(X_test)[:, 1]
test_auc = roc_auc_score(y_test, y_test_pred)
print("Weighted Ensemble Test AUC:", test_auc)

Weighted Ensemble Test AUC: 0.7649630634705262


#### The ensemble has successfully captured complementary strengths from both XGBoost and ExtraTreesClassifier.
- The weighted voting scheme gave XGBoost, the stronger individual performer, more influence on the final predictions.
- This AUC improvement suggests that the ensemble is robust and better at distinguishing between classes on unseen data compared to individual models.

### 5. Selecting the final model
- Choosing between logistic regression, ExtraTreesClassifier and XGBoost
- Training the final model
- Saving the model

##### Feature Importance for further insights

In [ ]:
# Average feature importances from both models
feature_importances = (
    np.array(best_xgb.feature_importances_) + np.array(best_et.feature_importances_)
) / 2

# Feature names
feature_names = dv.feature_names_

# Plot
indices = np.argsort(feature_importances)[::-1]
plt.figure(figsize=(10, 6))
plt.bar(range(len(feature_importances)), feature_importances[indices], align="center")
plt.xticks(range(len(feature_importances)), [feature_names[i] for i in indices], rotation=90)
plt.title("Ensemble Feature Importance")
plt.show()